In [5]:
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader   # For reading PDF files
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.tools import create_retriever_tool
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import convert_to_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.vectorstores import FAISS
import glob
from typing import List
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import faiss

# Aufgabenstellung

Sie bekommen Fachinformationen zu 3 Medikamenten aus dem Compendium und werden einen RAG-Prozess in einem Jupyter Notebook erstellen der Folgendes tut:
1. Alle PDFs werden als einen String eingelesen und Sie zeigen wie lange der String ist
2. Die PDFs werden nach Vorgaben in Chunks geteilt
3. Es wird ein FAISS Vector Store mit dem vorgegebenen Embedding Modell erstellt
4. Es wird ein Retriever konfiguriert, der mit der vorgegebenen Query getestet wird.
5. Es wird eine Antwortfunktion mit dem vorgegebenen LLM erstellt und mit 2 Fragen getestet.
6. Es werden 5 Testfragen von Chunks hergeleitet mit Hilfe eines LLMs (freie Wahl), die dann mit der gebauten Antwortfunktion beantwortet werden.


Die Dateien können auf Moodle[1] heruntergeladen werden. Achtung: Der Ordner beinhaltet mehr PDFs als Sie benötigen. Bitte verwenden Sie nur die 3 für Sie bestimmten Dateien (siehe unten).
Folgende Angaben brauchen Sie für deine Aufgabe:
- Sie arbeiten mit den PDF-Dateien: HEPCLUDEX.pdf , Abirateron Spirig.pdf, Azopt.pdf
- Sie wählen eine Chunk Size von 1500 und einen Overlap von 150 Zeichen
- Sie nehmen für die Embeddings das Modell: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 · Hugging Face
- Sie nehmen als Query für den Retriever 
    - «Wie soll Hepcludex gelagert werden?»
- Sie nehmen als LLM für die Antwortfunktion Qwen3-30B-A3B
- Sie nehmen als 2 Testfragen für die Antwortfunktion:
    - «Wie wird Abirateron normalerweise bei Erwachsenen dosiert?»
    - «Wer sollte Azopt nicht anwenden? Nenne mindestens eine wichtige Gegenanzeige.»

# Aufgabe 1 (PDF's auslesen und länge ausgeben)

In [4]:
DATA_DIR = "Data/*.pdf"  # Ordner mit Texten zu "Jugend, Politik, Umwelt & Partizipation"

text = ""

for pdf_path in glob.glob(DATA_DIR):
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += " " + page_text

len(text)

111567

# Aufgabe 2 (Split text in chunks)

In [6]:

splitter_a = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

chunks = splitter_a.split_text(text)

print(f"Anzahl Chunks: {len(chunks)}")


Anzahl Chunks: 84


# Aufgabe 3 (Faiss Vector Store erstellt)

In [ ]:
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("RAG-Setup abgeschlossen.")


os.makedirs("faiss", exist_ok=True)
faiss.write_index(index, "faiss/faiss_exam.index")
with open("faiss/chunks_exam.pkl", "wb") as f:
    pickle.dump(chunks, f)

RAG-Setup abgeschlossen.


# Aufgabe 4 (Retriever)

In [ ]:
def retrieve_texts(query: str, k: int, index, chunks: List[str], embedder) -> List[str]:
    """Gibt die Top-k ähnlichsten Chunks für eine Query zurück."""
    query_emb = embedder.encode([query], convert_to_numpy=True) 
    distances, indices = index.search(query_emb, k)
    return [chunks[i] for i in indices[0]]

test_query = "Wie soll Hepcludex gelagert werden?"
try:
    test_results = retrieve_texts(test_query, k=3, index=index, chunks=chunks, embedder=embedder)
    for i, r in enumerate(test_results, start=1):
        print(f"Chunk {i} (Ausschnitt):", r, "\n---\n")
except Exception as e:
    print("Fehler bei Testabfrage (erwartet, solange die Funktion noch nicht korrigiert ist):", e)

Chunk 1 (Ausschnitt): Hepcludex-Dosis Zu entnehmendes erforderliches Volumen von rekonstituiertem Hepcludex
1 mg 0,5 ml
1,5 mg 0,75 ml
2 mg 1 ml
Die Nadelspitze anschliessend von der Spritze abziehen. An diese Spritze eine Nadelspitze zur subkutanen Injektion anbringen und alle verbleibenden
Luftblasen vor der Injektion aus der Spritze entfernen.
Anweisungen zur Verabreichung
Verabreichung als subkutane Injektion in den Oberschenkel oder Unterbauch.
Wenn eine Dosis ausgelassen wurde, sollte die Verabreichung so bald wie möglich am selben Tag nachgeholt werden. Wenn es jedoch bereits fast Zeit für
die nächste Dosis ist, sollte die Dosis ausgelassen und der reguläre Zeitplan für die nächste Dosisgabe eingehalten werden. Es darf keine doppelte Dosis
verabreicht werden.
Injektionsstelle bei jeder Injektion wechseln.
Wichtig: Patienten sind daran zu erinnern, dass es wichtig ist, die Durchstechflaschen, Spritzen, Nadeln oder übrig bleibendes steriles Wasser für
Injektionszwecke nicht wieder

# Aufgabe 5: (Antwortfunktion mit LLM)

In [31]:
deepinfra_api_key = os.getenv("DEEPINFRA_API_KEY")


from openai import OpenAI
from dotenv import load_dotenv
import re

load_dotenv()

# Deepinfra-Client (OpenAI-kompatible API)
deepinfra_client = OpenAI(
    api_key=deepinfra_api_key,
    base_url="https://api.deepinfra.com/v1/openai",
)

model_name = "Qwen/Qwen3-30B-A3B"


def answer_query_a(query: str, k: int, index, chunks, embedder, client: OpenAI) -> str:
    """RAG-style answer: retrieve context and ask an LLM."""

    retrieved_chunks = retrieve_texts(query, k, index, chunks, embedder)
    context = ""
    for i, r in enumerate(retrieved_chunks, start=1):
        context += (f"<chunk nr='{i}'>{r}</chunk>\n---\n")
    


    messages = [
        {"role": "system", "content": f"Du bist ein Experte um medizischne Fragen aufgrund der dir zur verfügung gestellten Benutzungsanleitungen zu beantworten. Benutze nur die Informationen die in <chunk nr='n'> tags sind um die fragen zu beantworten."
         +"Die Antwort soll nur die finale Antwort beinhalten: entweder die extrahierte Information beinhalten oder eine Meldung, dass die Information in den chunks nicht gefunden werden konnte."
         +"Die Antwort soll in einem <answer> tag zurück gegeben werden"
         +f"{context}"},
        {"role": "user", "content": f"Die zu beantwortende Frage lautet: {query}"},
    ]

    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.8
    )

    answer =  completion.choices[0].message.content.strip()
    return re.findall(r'<answer>(.*?)</answer>', answer)[0]

# Quick manual test
query1 = "Wie wird Abirateron normalerweise bei Erwachsenen dosiert?"
answer1 = answer_query_a(query1, k=5, index=index, chunks=chunks, embedder=embedder, client=deepinfra_client)
print("RAG answer:", answer1)



query2 = "Wer sollte Azopt nicht anwenden? Nenne mindestens eine wichtige Gegenanzeige."
answer2 = answer_query_a(query2, k=5, index=index, chunks=chunks, embedder=embedder, client=deepinfra_client)
print("RAG answer:", answer2)

RAG answer: Die Informationen zur normalen Dosierung von Abirateron bei Erwachsenen sind in den bereitgestellten Chunks nicht enthalten.
RAG answer: Patienten mit schwerwiegender Nierenfunktionsstörung (C <30 ml/min) bzw. bei Patienten mit hyperchlorämischer Azidose sollten Azopt nicht anwenden, da es kontraindiziert ist.


# Aufgabe 6 (Testfragen von Chunks)

In [32]:

import random
def generate_questions_for_random_chunks(chunks, client: OpenAI, num_chunks: int = 5, max_retries: int = 2):
    selected_chunks = random.sample(chunks, num_chunks)
    qa_pairs = []

    for chunk in selected_chunks:
        messages = [{"role": "system", "content": "Formuliere auf der basis des gegeben textes in <chunk> tag eine gute Frage."+
            "Als output sollte nur die Frage zurückgegeben werden."
            +"<chunk>:\n" + chunk + "</chunk>"},
            {"role": "user", "content": "Gib eine Frage zurück für den gegebenen chunk"}
            ]
        

        question = None
        for attempt in range(max_retries):
            try:
                completion = client.chat.completions.create(
                    model="openai/gpt-oss-20b",
                    messages=messages
                )
                question = completion.choices[0].message.content.strip()
                if question:
                    break
            except Exception as e:
                print("Error while generating question, retrying...", e)

        if question is None:
            question = "Error: could not generate question."

        qa_pairs.append((chunk, question))

    return qa_pairs

questions = generate_questions_for_random_chunks(chunks, client=deepinfra_client, num_chunks=5, max_retries=2)
for i, (chunk, q) in enumerate(questions, 1):
        print(f"Frage {i}: {q}\n  für chunk: {chunk[:120]}...\n")
        answer = answer_query_a(q, 3, index, chunks, embedder, deepinfra_client)
        print(f"Antwort {i}: {answer}\n")



Frage 1: Welche Überwachungsintervalle gelten für Serum‑Transaminasen und Bilirubin bei Patienten, die mit einer reduzierten Dosis von 500 mg Abirateron HC neu behandelt werden, nachdem Hepatotoxizität aufgetreten ist?
  für chunk: wieder normalisiert haben (siehe «Warnhinweise und Vorsichtsmassnahmen» – «Hepatotoxizität»). Nach Rückgang der Leberfun...

Antwort 1: Bei Patienten, die erneut mit einer reduzierten Dosis von 500 mg Abirateron Spirig HC behandelt werden, sollten Serum-Transaminasen und Bilirubin über drei Monate mindestens alle zwei Wochen und anschließend einmal pro Monat überwacht werden.

Frage 2: Welches Kriterium muss laut den PCWG2‑Regeln erfüllt sein, damit eine radiologische Progression mittels zweier Knochenscans bestätigt wird, und wie definiert die Studie die Anzahl der neuen Läsionen im Vergleich zur ersten und zweiten Untersuchung?
  für chunk: der ersten 12 Wochen der Knochenscan ≥2 neuen Läsionen (PCWG2 Kriterien) zeigte und dies durch einen zweiten Knochens